In [58]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.neighbors import NearestNeighbors

In [49]:
# Carregar previsões dos modelos
feedforward_df = pd.read_csv('../feed_forward/previsao_confidence_feedforward.csv')
boosting_df = pd.read_csv('../boosting/resultados_previsao_boosting.csv')
decision_tree_df = pd.read_csv('../decision_tree/resultados_previsao_decisionTree.csv')

In [55]:
# Remover qualquer valor nulo nas colunas de latitude e longitude
boosting_df.dropna(subset=['latitude', 'longitude'], inplace=True)
decision_tree_df.dropna(subset=['latitude', 'longitude'], inplace=True)
feedforward_df.dropna(subset=['latitude', 'longitude'], inplace=True)

In [59]:
# Inicializa o modelo KNN para encontrar os vizinhos mais próximos
knn = NearestNeighbors(n_neighbors=1, metric='euclidean')

# Função para combinar os DataFrames com base em vizinhos mais próximos
def combine_with_knn(df1, df2):
    # Concatenar as latitudes e longitudes para realizar o KNN
    coords1 = df1[['latitude', 'longitude']].values
    coords2 = df2[['latitude', 'longitude']].values

    # Ajustar o modelo KNN para os dados do primeiro DataFrame (df1)
    knn.fit(coords1)

    # Encontrar os vizinhos mais próximos do segundo DataFrame (df2) no primeiro DataFrame
    _, indices = knn.kneighbors(coords2)

    # Associar as probabilidades de df1 com os dados de df2 usando os índices dos vizinhos
    df2['probabilidade_boosting'] = df1.iloc[indices.flatten()]['probabilidade'].values
    return df2


In [60]:
# 1. Combine boosting_df com decision_tree_df usando o KNN
combined_df = combine_with_knn(boosting_df, decision_tree_df)

In [61]:
# 2. Combine o resultado com feedforward_df
combined_df = combine_with_knn(combined_df, feedforward_df)

In [63]:
# Agora o DataFrame combinado terá as probabilidades dos três modelos
# 3. Criar a coluna de probabilidade final (ensemble) calculando a média das probabilidades
combined_df['probabilidade_ensemble'] = combined_df[['probabilidade', 'probabilidade', 'probabilidade']].mean(axis=1)

In [64]:
resultado_final = combined_df[['Data', 'latitude', 'longitude', 'probabilidade_ensemble']]

In [65]:
# Salvar o resultado em um CSV
resultado_final.to_csv('resultado_ensemble.csv', index=False)

CALCULANDO A ACURÁCIA

In [68]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

In [69]:
# Suponha que você tenha os valores reais das probabilidades (por exemplo, da sua base de dados real)
# Aqui vamos gerar um exemplo de dados reais para ilustrar. Substitua isso pelos seus valores reais.

# Suponha que 'probabilidade_real' seja a coluna de probabilidades reais que você possui
# Para o exemplo, vamos gerar uma variável fake de probabilidades reais
real_probabilities = np.random.rand(len(resultado_final)) * 100  # Aqui substitua pelos seus valores reais

# As probabilidades previstas do seu modelo de ensemble
predicted_probabilities = resultado_final['probabilidade_ensemble']

In [70]:
# 1. Cálculo do Erro Médio Absoluto (MAE)
mae = mean_absolute_error(real_probabilities, predicted_probabilities)
print(f"Erro Médio Absoluto (MAE): {mae:.4f}")

Erro Médio Absoluto (MAE): 33.1754


In [71]:
# 2. Cálculo do Erro Quadrático Médio (RMSE)
rmse = np.sqrt(mean_squared_error(real_probabilities, predicted_probabilities))
print(f"Erro Quadrático Médio (RMSE): {rmse:.4f}")

Erro Quadrático Médio (RMSE): 40.7538


In [72]:
# 3. Cálculo do Coeficiente de Determinação (R²)
r2 = r2_score(real_probabilities, predicted_probabilities)
print(f"Coeficiente de Determinação (R²): {r2:.4f}")

Coeficiente de Determinação (R²): -0.9934
